In [56]:
## inaguaral pybaseball notebook !! 
#imports
from pybaseball import statcast
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import logging
import polars as pl
%matplotlib inline

In [57]:
#find playerid lookup of zack wheeler
pitcher= playerid_lookup("wheeler","zack")
pitcherid = pitcher.loc[0,"key_mlbam"]
pitcherid = float(pitcherid)
#pull zack wheeler's pitch data from the 2025 season
wheeler_pitches = statcast_pitcher('2025-04-02', '2025-10-09', pitcherid)
wheeler_pitches.shape

Gathering Player Data


(2297, 118)

In [58]:
#convert wheeler_pitches from pandas dataframe to polars dataframe
wheeler_pitches = pl.from_pandas(wheeler_pitches)
#create new column which is the unique ID of the at bat
wheeler_pitches = wheeler_pitches.with_columns(
    pl.concat_str([pl.col("game_pk").cast(pl.Utf8), pl.lit("_"), pl.col("at_bat_number").cast(pl.Utf8)]).alias("ab_id")
)


In [59]:
#count unique number of ab_ids so that we can have an idea of how much data we should have
abs = wheeler_pitches["ab_id"].n_unique()
print(f"there are {abs} at-bats in these data")


there are 568 at-bats in these data


In [ ]:
#filter by the last or second to last pitches in a given at bat
pitches_ofinterest = wheeler_pitches.filter(
    pl.col("pitch_number") == (pl.col("pitch_number").max().over("ab_id")) | (pl.col("pitch_number").max().over("ab_id")-1) 
)
# filter out if there is only one pitch in the at bat


there are 144.5 pitch combinations of interest in these data


(289, 119)

In [ ]:

last_pitches = wheeler_pitches.filter(
    pl.col("pitch_number") == pl.col("pitch_number").max().over("ab_id")
)

event_counts = last_pitches.group_by("description").agg(
    pl.count().alias("count")
).sort("count", descending=True)

event_counts

In [ ]:
# pitch tunneling, for Zack's most common pitch combinations, what is the most location differential to maximize deception?
#identify two pitch sequences that he uses within at-bats, might want to use the one that gives you a whiff or contact on
#the last pitch of the at bat
# Create a column with the next pitch type in the at-bat
pitch_sequences = wheeler_pitches.sort(['ab_id', 'pitch_number']).with_columns(
    next_pitch_type = pl.col('pitch_type').shift(-1).over('ab_id')
)

# Filter out the last pitch of each at-bat (where next_pitch_type is null)
pitch_sequences = pitch_sequences.filter(pl.col('next_pitch_type').is_not_null())

# Create a sequence label
pitch_sequences = pitch_sequences.with_columns(
    sequence = pl.concat_str([
        pl.col('pitch_type'),
        pl.lit('→'),
        pl.col('next_pitch_type')
    ])
)

# Count each two-pitch sequence
sequence_counts = pitch_sequences.group_by('sequence').agg(
    pl.count().alias('count')
).sort('count', descending=True)

In [ ]:
#calculate the percent of the total that is each of these rows
sequence_counts = sequence_counts.with_columns(
    (pl.col("count") / pl.col("count").sum()).alias("pc_oftotal")
)
#filter sequence counts to anything more than 5%


In [ ]:
# keep only top 5 most frequent two-pitch sequences
top5_sequences = sequence_counts.sort('count', descending=True).head(5)
top5_sequences

In [ ]:
#now we need to find these pitches in the original data and pull out the location
#information from each pitch
